In [ ]:
import cv2
import random
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from PIL import Image

def pil_to_tensor(pil_image):
  transform = transforms.ToTensor()
  tensor_image = transform(pil_image)
  return tensor_image

def tensor_to_pil(tensor_image):
    transform = transforms.ToPILImage()
    pil_image = transform(tensor_image)
    return pil_image

def load_images_from_folder(folder):
  images = []
  for filename in os.listdir(folder):
    img_path = os.path.join(folder, filename)
    if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
      try:
        img = Image.open(img_path)
        images.append(img)
      except Exception as e:
        print(f"Error loading image {img_path}: {e}")
  return images

def draw_boundary_on_image(image_tensor, mask_tensor, boundary_color=(0, 0, 255), boundary_thickness=10):
    if mask_tensor.dtype != torch.uint8:
        mask_tensor = (mask_tensor * 255).to(torch.uint8)

    if image_tensor.dtype != torch.uint8:
        image_tensor = (image_tensor * 255).to(torch.uint8)

    if len(mask_tensor.shape) == 3:
        mask_tensor = mask_tensor[0, :, :]

    mask_np = mask_tensor.numpy().astype(np.uint8)
    contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_np = image_tensor.permute(1, 2, 0).numpy().copy()  # [C, H, W] -> [H, W, C]
    cv2.drawContours(image_np, contours, -1, boundary_color, boundary_thickness)
    output_image_tensor = torch.from_numpy(image_np).permute(2, 0, 1)  # 다시 [C, H, W]로 변경
    
    return output_image_tensor

In [ ]:
!git clone https://github.com/ByeongHyunPak/omni-proj.git
!pip install tensorboardX

import os
os.chdir('/content/omni-proj/omni_proj')

imgs_folder = '/content/omni-proj/imgs/erps' 
images = load_images_from_folder(imgs_folder)

for img in images:
    img = pil_to_tensor(img)
    print(img.shape)

In [ ]:
import torch
import numpy as np

def rodrigues_torch(rvec):
    theta = torch.norm(rvec)
    if theta < 1e-6:
        return torch.eye(3, device=rvec.device)

    rvec = rvec / theta
    K = torch.tensor([[0, -rvec[2], rvec[1]],
                      [rvec[2], 0, -rvec[0]],
                      [-rvec[1], rvec[0], 0]], device=rvec.device)
    R = torch.eye(3, device=rvec.device) + torch.sin(theta) * K + (1 - torch.cos(theta)) * torch.matmul(K, K)
    return R  

def gridy2x_per2erp_cuda(gridy, HWy, HWx, THETA, PHI, FOVy, FOVx, device='cuda'):
    H, W, h, w = *HWy, *HWx
    hFOVy, wFOVy = FOVy * float(H) / W, FOVy
    hFOVx, wFOVx = FOVx * float(h) / w, FOVx

    # gridy2x
    ### onto sphere
    gridy = gridy.reshape(-1, 2).float()
    lat = gridy[:, 0] * np.pi / 2
    lon = gridy[:, 1] * np.pi

    z0 = torch.sin(lat)
    y0 = torch.cos(lat) * torch.sin(lon)
    x0 = torch.cos(lat) * torch.cos(lon)
    gridy = torch.stack((x0, y0, z0), dim=-1).double()

    ### rotation
    y_axis = torch.tensor([0.0, 1.0, 0.0], device=device, dtype=torch.float64)
    z_axis = torch.tensor([0.0, 0.0, 1.0], device=device, dtype=torch.float64)
    R1 = rodrigues_torch(z_axis * np.radians(THETA))
    R2 = rodrigues_torch(torch.matmul(R1, y_axis) * np.radians(PHI))

    R1_inv = torch.inverse(R1)
    R2_inv = torch.inverse(R2)

    gridy = torch.mm(R2_inv, gridy.permute(1, 0)).permute(1, 0)
    gridy = torch.mm(R1_inv, gridy.permute(1, 0)).permute(1, 0)

    ### sphere to gridx
    z0 = gridy[:, 2] / gridy[:, 0]
    y0 = gridy[:, 1] / gridy[:, 0]
    gridx = torch.stack((z0, y0), dim=-1).float()

    # masky
    mask = torch.where(torch.abs(gridx) > 1, 0, 1)
    mask = mask[:, 0] * mask[:, 1]
    mask *= torch.where(gridy[:, 0] < 0, 0, 1)

    return gridx.to(torch.float32), mask.to(torch.float32)

def gridy2x_erp2per_cuda(gridy, HWy, HWx, THETA, PHI, FOVy, FOVx, device='cuda'):
    H, W, h, w = *HWy, *HWx
    hFOVy, wFOVy = FOVy * float(H) / W, FOVy
    hFOVx, wFOVx = FOVx * float(h) / w, FOVx
    
    # gridy2x
    ### onto sphere
    gridy = gridy.reshape(-1, 2).float()
    gridy[:, 0] *= np.tan(np.radians(hFOVy / 2.0))
    gridy[:, 1] *= np.tan(np.radians(wFOVy / 2.0))
    gridy = gridy.double().flip(-1)
    
    x0 = torch.ones(gridy.shape[0], 1, device=device)
    gridy = torch.cat((x0, gridy), dim=-1)
    gridy /= torch.norm(gridy, p=2, dim=-1, keepdim=True)
    
    ### rotation
    y_axis = torch.tensor([0.0, 1.0, 0.0], device=device, dtype=torch.float64)
    z_axis = torch.tensor([0.0, 0.0, 1.0], device=device, dtype=torch.float64)
    R1 = rodrigues_torch(z_axis * np.radians(THETA))
    R2 = rodrigues_torch(torch.matmul(R1, y_axis) * np.radians(PHI))

    gridy = torch.mm(R1, gridy.permute(1, 0)).permute(1, 0)
    gridy = torch.mm(R2, gridy.permute(1, 0)).permute(1, 0)

    ### sphere to gridx
    lat = torch.arcsin(gridy[:, 2]) / np.pi * 2
    lon = torch.atan2(gridy[:, 1] , gridy[:, 0]) / np.pi
    gridx = torch.stack((lat, lon), dim=-1)

    # masky
    mask = torch.where(torch.abs(gridx) > 1, 0, 1)
    mask = mask[:, 0] * mask[:, 1]

    return gridx.to(torch.float32), mask.to(torch.float32)

In [ ]:
import utils

def erp2per_cuda(hr_erp_img, THETA, PHI, FOVy=90, FOVx=360, HWy=(512, 512), device='cuda'):
    hr_erp_img = pil_to_tensor(hr_erp_img).to(device)
    HWx = hr_erp_img.shape[-2:]

    gridy = utils.make_coord(HWy).to(device)
    gridy2x, masky = gridy2x_erp2per_cuda(
        gridy, HWy, HWx, THETA, PHI, FOVy, FOVx, device)
    gridy2x = gridy2x.view(*HWy, 2)

    inp = F.grid_sample(hr_erp_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        padding_mode='reflection',
                        align_corners=False).clamp_(0, 1)[0]

    gridx = utils.make_coord(HWx, flatten=False).to(device)
    gridx2y, maskx = gridy2x_per2erp_cuda(
        gridx, HWx, HWy, THETA, PHI, FOVx, FOVy, device)
    
    maskx = maskx.view(1, *HWx)
    valid_hr_erp_img = hr_erp_img * maskx
    
    return inp, valid_hr_erp_img, maskx

def per2erp_cuda(hr_pers_img, THETA, PHI, FOVy=360, FOVx=90, HWy=(512, 1024), device='cuda'):
    hr_pers_img = pil_to_tensor(hr_pers_img).to(device)
    HWx = hr_pers_img.shape[-2:]

    gridy = utils.make_coord(HWy).to(device)
    gridy2x, masky = gridy2x_per2erp_cuda(
        gridy, HWy, HWx, THETA, PHI, FOVy, FOVx, device)
    gridy2x = gridy2x.view(*HWy, 2)
    masky = masky.view(1, *HWy)

    inp = F.grid_sample(hr_pers_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        padding_mode='reflection',
                        align_corners=False).clamp_(0, 1)[0]
    inp = inp * masky

    gridx = utils.make_coord(HWx, flatten=False).to(device)
    gridx2y, maskx = gridy2x_erp2per_cuda(
        gridx, HWx, HWy, THETA, PHI, FOVx, FOVy, device)
    
    maskx = maskx.view(1, *HWx)
    valid_hr_pers_img = hr_pers_img * maskx
    
    return inp, valid_hr_pers_img, maskx

In [ ]:
device = 'cuda'
hr_erp_img = images[0]
display(hr_erp_img)

# ERP to Perspective
THETA = random.uniform(-135, 135)
PHI = random.uniform(-45, 45)

pers_img, valid_hr_erp_img, _ = erp2per_cuda(hr_erp_img, THETA, PHI, FOVy=90, FOVx=360, device=device)

display(tensor_to_pil(valid_hr_erp_img))
display(tensor_to_pil(pers_img))

# Pers to ERP
erp_img, valid_lr_pers_img, _ = per2erp_cuda(tensor_to_pil(pers_img), THETA, PHI, FOVy=360, FOVx=90, device=device)

display(tensor_to_pil(valid_lr_pers_img))
display(tensor_to_pil(erp_img))